## TODO:

- build an exploratory data analysis
- use the one from MC1 as reference: https://github.com/upb-lea/hardcore-magnet-challenge/blob/main/notebooks/wk-1.1-eda.ipynb

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

import pathlib
import glob

import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import jax
import jax.numpy as jnp
import equinox as eqx

In [ ]:
from mc2.utils.data_inspection import (
    get_available_material_names,
    get_file_overview,
    load_and_process_single_from_full_file_overview
)
from mc2.data_management import FrequencySet, MaterialSet, DataSet

## load data from pickle:

In [ ]:
dataset = DataSet.load_from_file(pathlib.Path("../../data/processed") / "ten_mat_data.pickle")

In [ ]:
dataset

In [ ]:
dataset.filter_frequencies([80_000])
# dataset.filter_temperatures([25, 50])

## For the first material "3C90":

In [ ]:
material_set_list = [
    FrequencySet.from_dict(
        load_and_process_single_from_full_file_overview(
            full_file_overview,
            material_name="3C90",
            data_type=["B", "T", "H"],
            frequency=[frequency],
        )
    )
    for frequency in frequencies
]

In [ ]:
material_set = MaterialSet(
    material_name=material_set_list[0].material_name,
    frequency_sets=material_set_list,
    frequencies=frequencies
)
material_set

In [ ]:
full_set_lengths = []
for freq_set in material_set:
    n_sequences, sequence_length = freq_set.B.shape
    full_set_length = n_sequences * sequence_length

    full_set_lengths.append(full_set_length)

plt.suptitle("Datapoints per frequency")
plt.plot(frequencies / 1e3, np.array(full_set_lengths) / 1e6)
plt.xlabel("f in kHz")
plt.ylabel("# of datapoints in M")

In [ ]:
print("unique temperatures per frequency")
unique_temperatures = {int(frequency): jnp.unique(freq_set.T) for frequency, freq_set in zip(material_set.frequencies, material_set)}
display(unique_temperatures)

for freq_set in material_set:
    plt.suptitle("Temperatures per sequence")
    plt.plot(freq_set.T, label=str(int(freq_set.frequency / 1e3)) + " kHz")
    plt.legend()

plt.xlabel("sequence index")
plt.ylabel("T in °C")
plt.show()

There are always the temperature levels 25, 50, and 70 for all frequencies (at least for "3C90")

In [ ]:
full_set_lengths = []
for freq_set in material_set:

    n_sequences, sequence_length = freq_set.B.shape
    full_set_length = n_sequences * sequence_length

    full_set_lengths.append(full_set_length)

plt.suptitle("Datapoints per temperature")
plt.plot(temperatures, n_data_points)
plt.xlabel("T in °C")
plt.ylabel("# of datapoints")